In [1]:
import sys
sys.path.insert(0, '../')
from callsetmerger.callsetmerger import Readers
from callsetmerger.recordcluster import convert_type_to_idx
from callsetmerger.recordcluster import GetVcfTypesKey
import trtools.utils.tr_harmonizer as trh
from trtools.utils.utils import GetCanonicalMotif
import itertools
import pandas as pd
import upsetplot
import matplotlib.pyplot as plt

In [2]:
eh_index= list(GetVcfTypesKey()).index(trh.VcfTypes.eh)
gs_index= list(GetVcfTypesKey()).index(trh.VcfTypes.gangstr)
ad_index= list(GetVcfTypesKey()).index(trh.VcfTypes.advntr)
hp_index= list(GetVcfTypesKey()).index(trh.VcfTypes.hipstr)

gangstr_only_recs = []
hipstr_only_recs = []
readers = Readers('/home/nmmsv/workspace/analysis/callset_merging/input/advntr.chr21.sorted.vcf.gz,/home/nmmsv/workspace/analysis/callset_merging/input/hipstr.chr21.sorted.vcf.gz,/home/nmmsv/workspace/analysis/callset_merging/input/gangstr.chr21.sorted.vcf.gz,/home/nmmsv/workspace/analysis/callset_merging/input/eh.chr21.sorted.vcf.gz'.split(","))
while not readers.done:
    # Get mergeable calls
    rc_list = readers.getMergableCalls()
    for rc in rc_list:
        # Only gangstr
        if not rc.vcf_types[eh_index] and \
            rc.vcf_types[gs_index] and \
            not rc.vcf_types[ad_index] and \
            not rc.vcf_types[hp_index]:
            rec = rc.record_objs[0]
            gangstr_only_recs.append(rec)
        # Only hipstr
        if not rc.vcf_types[eh_index] and \
            not rc.vcf_types[gs_index] and \
            not rc.vcf_types[ad_index] and \
            rc.vcf_types[hp_index]:
            rec = rc.record_objs[0]
            hipstr_only_recs.append(rec)
    # Move on
    readers.goToNext()

TypeError: 'OverlappingRegion' object is not iterable

In [6]:
hipstr_only_pos = []
gangstr_only_pos = []
for rec in hipstr_only_recs:
    hipstr_only_pos.append((rec.hm_record.vcfrecord.POS,rec.canonical_motif))

for rec in gangstr_only_recs:
    gangstr_only_pos.append((rec.hm_record.vcfrecord.POS,rec.canonical_motif))

In [7]:
n = 0
for pg in gangstr_only_pos:
    for ph in hipstr_only_pos:
        if abs(pg[0] - ph[0]) < 10:
            if pg[1] == ph[1]:
                n = n + 1
                print('GangSTR: ', pg, '\tHipSTR: ', ph)
n

GangSTR:  (20917849, 'AGAT') 	HipSTR:  (20917843, 'AGAT')
GangSTR:  (25690272, 'AAAG') 	HipSTR:  (25690265, 'AAAG')
GangSTR:  (29126456, 'AAAC') 	HipSTR:  (29126448, 'AAAC')
GangSTR:  (34305937, 'A') 	HipSTR:  (34305934, 'A')
GangSTR:  (38726544, 'AAGG') 	HipSTR:  (38726536, 'AAGG')
GangSTR:  (40669591, 'A') 	HipSTR:  (40669587, 'A')
GangSTR:  (44469088, 'AGGGG') 	HipSTR:  (44469083, 'AGGGG')
GangSTR:  (46090119, 'ACC') 	HipSTR:  (46090117, 'ACC')
GangSTR:  (46164003, 'AAAC') 	HipSTR:  (46164000, 'AAAC')


9

In [8]:
hipstr_only_closest_gng =[]
# Find the closest gangstr pos for each hipstr pos
n = 0
for ph in hipstr_only_pos:
    closest_dist = 10000000
    closest = []
    for pg in gangstr_only_pos:
        if abs(pg[0] - ph[0]) < closest_dist:
            closest_dist = abs(pg[0] - ph[0])
            closest = pg
    if 1:#GetCanonicalMotif(ph[1]) == GetCanonicalMotif(closest[1]):
        n = n + 1
        print(ph, closest, closest_dist)
        # hipstr_only_closest_gng.append((ph, closest))
n

'A') (44254828, 'AAC') 678
(44257378, 'A') (44257694, 'AAAT') 316
(44261955, 'CCCCG') (44262266, 'ACCCC') 311
(44264118, 'A') (44262511, 'ACCC') 1607
(44264447, 'AAAAAG') (44262511, 'ACCC') 1936
(44265035, 'A') (44262511, 'ACCC') 2524
(44265562, 'A') (44262511, 'ACCC') 3051
(44270162, 'A') (44275183, 'ACCTGCCCC') 5021
(44271133, 'AAAAG') (44275183, 'ACCTGCCCC') 4050
(44276917, 'A') (44277091, 'AATT') 174
(44278712, 'AAAAT') (44277091, 'AATT') 1621
(44280278, 'A') (44277091, 'AATT') 3187
(44284159, 'AGGC') (44285190, 'AAGG') 1031
(44299042, 'C') (44298784, 'AAAAG') 258
(44311354, 'AC') (44308560, 'A') 2794
(44316693, 'AC') (44316573, 'ACACCC') 120
(44326629, 'C') (44324659, 'AGCCC') 1970
(44337084, 'A') (44334745, 'AATG') 2339
(44343669, 'A') (44344383, 'A') 714
(44359588, 'A') (44363363, 'AAAC') 3775
(44359722, 'AT') (44363363, 'AAAC') 3641
(44362349, 'A') (44363363, 'AAAC') 1014
(44362513, 'AAAG') (44363363, 'AAAC') 850
(44362778, 'A') (44363363, 'AAAC') 585
(44368416, 'A') (44368113,

7733